### Simulatore dati e streaming verso Event Hub
v3.0

In [1]:
import numpy as np
import json
import time
import datetime
import uuid
import collections

In [2]:
!pip install azure.eventhub

  Using cached https://files.pythonhosted.org/packages/02/e9/e433b9e9214f872e7813851dee5c7266d69c305ad483aee391d6107db428/azure_eventhub-1.3.1-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from azure.eventhub import EventHubClient, Sender, EventData

Some notes:

- The batches are defined below in Batches definition (they could be generated in a loop by indicanting just the number of batches)
- All batches involve the same machine/sensors (this can be easily modified by generating the random GUID's inside the loop)
- The duration of the simulation is defined by batch_planning and batch_duration

In [4]:
# Function temperature sensor
def sensor_temp(min, max, id, batch, datetimeID):
    y = round(np.random.uniform(min, max), 2)
    timestamp = datetime.datetime.now()
    data = [('batch-id', batch['ID_prefix'] + datetimeID),
            ('channel-id', id),
            ('val', y),
            ('time', timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
            ]
    data = collections.OrderedDict(data)
    return data

# Function pressure sensor
def sensor_press(min, max, id, batch, datetimeID):
    y = round(np.random.uniform(min, max), 2)
    timestamp = datetime.datetime.now()
    data = [('batch-id', batch['ID_prefix'] + datetimeID),
            ('channel-id', id),
            ('val', y),
            ('time', timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
            ]
    data = collections.OrderedDict(data)
    return data

In [32]:
# Batches definition
batches = {1: {'batch_description': 'injection piece A1', 'ID_prefix': 'INJPCA1_'},
                2: {'batch_description': 'injection piece A2', 'ID_prefix': 'INJPCA2_'},
                3: {'batch_description': 'injection piece A3', 'ID_prefix': 'INJPCA3_'},
                4: {'batch_description': 'injection piece A4', 'ID_prefix': 'INJPCA4_'},
                5: {'batch_description': 'injection piece A5', 'ID_prefix': 'INJPCA5_'}}

# Batches scheduling
batch_planning = [1, 2, 3, 4, 5] #[1, 2, 3, 1, 2, 3, 1, 1] # sequence of batches = keys in batches dictionary
N_batches = len(batch_planning) # total number of batches

# Batches duration (choose batch_duration = batch_duration_fix OR batch_duration = batch_duration_random)
minutes_min, minutes_max = 1, 3 # range of batch duration in minutes (for random)
minutes_fix = 1 # batch duration in minutes (for fix)
batch_duration_fix = [minutes_fix]*N_batches
batch_duration_random = np.random.randint(minutes_min, minutes_max + 1, N_batches).tolist()
batch_duration = batch_duration_fix #batch_duration_random #batch_duration_fix

# ID producers: list of predefined ID's or randomly generated GUID's
producer_ID = ['T1', 'P1', 'T2', 'P2', 'T3', 'P3'] # list of predefined ID's
N_channels = len(producer_ID)

if producer_ID == []:
    # ID producers (sensor channel identifiers, 36-digits randomly generated GUID according to standard RFC-4122)
    N_channels = 4
    producer_ID = [str(uuid.uuid4()) for i in range(N_channels)] # list of ID's, example 'f66aedac-9749-4491-837a-f0cb623cc2df'

# Sampling frequency (unique for all sensors)
sampling_freq = 1 # measurements/sec

In [6]:
# INFORMATION ABOUT THE CREATED EVENT HUBS NAMESPACE AND EVENT HUB
'''
ADDRESS = 'xxxxx' # example 'amqps://eventhub1-idaq-test6.servicebus.windows.net/eventhub1'
USER = 'RootManageSharedAccessKey'
KEY = 'xxxxx' # Primary key
client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
sender = client.add_sender(partition="0")
client.run()
'''

'\nADDRESS = \'xxxxx\' # example \'amqps://eventhub1-idaq-test6.servicebus.windows.net/eventhub1\'\nUSER = \'RootManageSharedAccessKey\'\nKEY = \'xxxxx\' # Primary key\nclient = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)\nsender = client.add_sender(partition="0")\nclient.run()\n'

In [33]:
# Loop generating measurements
start_time = time.time()
datetimeID = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")

batch_order = 0 # element of batch_planning list

while(batch_order < N_batches):

    N_batch = batch_planning[batch_order]
    
    for producer in zip(producer_ID[::2], producer_ID[1::2]):
        
        # First sensor data
        data = sensor_temp(20, 30, producer[0], batches[N_batch], datetimeID) # specify range for measured value
        datax = json.dumps(data)
        print(datax)
        #sender.send(EventData(datax))
        
        # Second sensor data
        data = sensor_press(120, 160, producer[1], batches[N_batch], datetimeID) # specify range for measured value
        datax = json.dumps(data)
        print(datax)
        #sender.send(EventData(datax))
        
        # Add more sensors here, maximum number of sensors = N_channels
    
    time.sleep(1 / sampling_freq)

    elapsed_time = (time.time() - start_time) / 60 # minutes
    
    if elapsed_time > batch_duration[batch_order]: # in minutes
        batch_order += 1
        start_time = time.time()
        datetimeID = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")

#client.stop()

{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 29.88, "time": "2019-05-10 13:06:34.625"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 135.03, "time": "2019-05-10 13:06:34.625"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 27.58, "time": "2019-05-10 13:06:34.626"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P2", "val": 135.29, "time": "2019-05-10 13:06:34.626"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T3", "val": 27.7, "time": "2019-05-10 13:06:34.626"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P3", "val": 137.91, "time": "2019-05-10 13:06:34.626"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 27.8, "time": "2019-05-10 13:06:35.628"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 142.34, "time": "2019-05-10 13:06:35.628"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 20.02, "time": "2019-05-10 13:06:35.628"}
{"batch-

{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 26.7, "time": "2019-05-10 13:06:47.659"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 128.37, "time": "2019-05-10 13:06:47.660"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 29.66, "time": "2019-05-10 13:06:47.660"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P2", "val": 133.85, "time": "2019-05-10 13:06:47.660"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T3", "val": 27.41, "time": "2019-05-10 13:06:47.660"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P3", "val": 156.04, "time": "2019-05-10 13:06:47.660"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 26.32, "time": "2019-05-10 13:06:48.662"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 138.42, "time": "2019-05-10 13:06:48.662"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 28.27, "time": "2019-05-10 13:06:48.662"}
{"batch

{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 28.09, "time": "2019-05-10 13:07:00.690"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 144.38, "time": "2019-05-10 13:07:00.690"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 25.0, "time": "2019-05-10 13:07:00.690"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P2", "val": 147.28, "time": "2019-05-10 13:07:00.690"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T3", "val": 21.35, "time": "2019-05-10 13:07:00.690"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P3", "val": 136.37, "time": "2019-05-10 13:07:00.691"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 26.49, "time": "2019-05-10 13:07:01.692"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 156.04, "time": "2019-05-10 13:07:01.692"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 21.6, "time": "2019-05-10 13:07:01.693"}
{"batch-

{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 29.93, "time": "2019-05-10 13:07:13.723"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 125.71, "time": "2019-05-10 13:07:13.724"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 20.68, "time": "2019-05-10 13:07:13.724"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P2", "val": 140.52, "time": "2019-05-10 13:07:13.725"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T3", "val": 21.45, "time": "2019-05-10 13:07:13.725"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P3", "val": 130.59, "time": "2019-05-10 13:07:13.726"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 28.24, "time": "2019-05-10 13:07:14.727"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 134.03, "time": "2019-05-10 13:07:14.727"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 28.58, "time": "2019-05-10 13:07:14.727"}
{"batc

{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 21.44, "time": "2019-05-10 13:07:26.756"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 130.26, "time": "2019-05-10 13:07:26.756"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 22.36, "time": "2019-05-10 13:07:26.756"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P2", "val": 140.56, "time": "2019-05-10 13:07:26.756"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T3", "val": 24.01, "time": "2019-05-10 13:07:26.756"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P3", "val": 151.79, "time": "2019-05-10 13:07:26.756"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T1", "val": 23.56, "time": "2019-05-10 13:07:27.758"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "P1", "val": 143.76, "time": "2019-05-10 13:07:27.758"}
{"batch-id": "INJPCA1_2019_05_10_13_06", "channel-id": "T2", "val": 25.17, "time": "2019-05-10 13:07:27.758"}
{"batc

...

{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 21.56, "time": "2019-05-10 13:10:42.228"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 126.49, "time": "2019-05-10 13:10:42.228"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 24.4, "time": "2019-05-10 13:10:42.229"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P2", "val": 139.97, "time": "2019-05-10 13:10:42.229"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T3", "val": 20.41, "time": "2019-05-10 13:10:42.229"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P3", "val": 147.61, "time": "2019-05-10 13:10:42.229"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 26.99, "time": "2019-05-10 13:10:43.230"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 139.48, "time": "2019-05-10 13:10:43.230"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 26.05, "time": "2019-05-10 13:10:43.231"}
{"batch

{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 24.62, "time": "2019-05-10 13:10:55.262"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 137.54, "time": "2019-05-10 13:10:55.262"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 25.88, "time": "2019-05-10 13:10:55.263"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P2", "val": 128.49, "time": "2019-05-10 13:10:55.263"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T3", "val": 23.29, "time": "2019-05-10 13:10:55.263"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P3", "val": 132.62, "time": "2019-05-10 13:10:55.263"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 21.45, "time": "2019-05-10 13:10:56.266"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 142.96, "time": "2019-05-10 13:10:56.266"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 29.49, "time": "2019-05-10 13:10:56.266"}
{"batc

{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 24.05, "time": "2019-05-10 13:11:08.292"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 129.55, "time": "2019-05-10 13:11:08.292"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 20.81, "time": "2019-05-10 13:11:08.292"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P2", "val": 128.22, "time": "2019-05-10 13:11:08.292"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T3", "val": 22.86, "time": "2019-05-10 13:11:08.292"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P3", "val": 132.41, "time": "2019-05-10 13:11:08.293"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 22.53, "time": "2019-05-10 13:11:09.294"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 124.64, "time": "2019-05-10 13:11:09.294"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 29.98, "time": "2019-05-10 13:11:09.294"}
{"batc

{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 22.68, "time": "2019-05-10 13:11:21.325"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 143.2, "time": "2019-05-10 13:11:21.325"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 25.65, "time": "2019-05-10 13:11:21.325"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P2", "val": 124.83, "time": "2019-05-10 13:11:21.325"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T3", "val": 23.06, "time": "2019-05-10 13:11:21.326"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P3", "val": 138.48, "time": "2019-05-10 13:11:21.326"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 28.21, "time": "2019-05-10 13:11:22.327"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 127.5, "time": "2019-05-10 13:11:22.328"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 25.9, "time": "2019-05-10 13:11:22.328"}
{"batch-i

{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T1", "val": 21.01, "time": "2019-05-10 13:11:34.354"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P1", "val": 159.56, "time": "2019-05-10 13:11:34.354"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T2", "val": 22.25, "time": "2019-05-10 13:11:34.354"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P2", "val": 140.04, "time": "2019-05-10 13:11:34.354"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "T3", "val": 21.01, "time": "2019-05-10 13:11:34.355"}
{"batch-id": "INJPCA5_2019_05_10_13_10", "channel-id": "P3", "val": 159.33, "time": "2019-05-10 13:11:34.355"}
